In [1]:
# unit test
import unittest

from PIL import Image, ImageDraw

from torch.utils.data import Dataset, DataLoader

import torch


import numpy as np

# "helper" needs to be part of sys path
import sys
sys.path.insert(0, "helper")
sys.path.insert(0, "/helper")
sys.path.insert(0, "./helper")
sys.path.insert(0, "../helper")
print(sys.path)

%load_ext autoreload
%autoreload 2

# own module
from sampler import *

['../helper', './helper', '/helper', 'helper', 'C:\\Users\\Prinzessin\\projects\\decentnet\\datasceyence\\examples', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\python39.zip', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\DLLs', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta', '', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Prinzessin\\.ipython']


In [2]:
class DS(Dataset):
    def __init__(self):
        super(DS, self).__init__()
        
        self.data  = [11, 12, 13, 14, 15, 16, 17, 18, 19, 
                      21, 22, 23, 24, 25, 26, 27, 28, 29, -21, -22, -23, -24,
                      71, 72, 73, 74, 75, 76,
                      #42, 43
                     ]
        self.lbl   = [1,  1,  1,  1,  1,  1,  1,  1,  1,  
                      2,  2,  2,  2,  2,  2,  2,  2,  2,   2,   2,   2,   2,
                      7,  7,  7,  7,  7,  7,
                      #0, 0
                     ]
        
    def get_labels(self):
        return list(self.lbl)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        
        item = { "img" : self.data[index],
                 "lbl" : self.lbl[index]
               }
        
        return item   

In [3]:
class UnitTestLoss(unittest.TestCase):   
    
    def test_mixed_batch_sampler(self):
        
        
        # if 1+: taking care of balance in batches in epoch
        # if smaller than 1: not taking care of balance accross batches in epoch
        n_samples_per_class_per_batch = 3 # 2 # 0.5
        
        ds = DS()

        print("-"*5, "item", "-"*5)
        print(ds.__getitem__(2))

        print("-"*5, "label", "-"*5)
        print(ds.get_labels())


        mbs = MixedBatchSampler(ds.get_labels(), n_samples_per_class_per_batch=n_samples_per_class_per_batch)

        print("-"*5, "idx", "-"*5)
        print(list(mbs))


        train_loader = DataLoader(ds, batch_sampler=mbs)

        print("")
        print("="*10 ,"train loader", "="*10)
        print("")

        for idx, item in enumerate(train_loader):

            print("-"*5, "batch", "-"*5)
            print(item["img"])
            print(item["lbl"])
            
            label_counter = torch.nn.functional.one_hot(item["lbl"]).sum(dim = 0)
            for c in label_counter:
                if n_samples_per_class_per_batch < 1:
                    n_samples_per_class_per_batch = 1
                if c != 0: # exclude empty classes
                    self.assertEqual(c, n_samples_per_class_per_batch)
            
            
            

In [4]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.009s

OK


----- item -----
{'img': 13, 'lbl': 1}
----- label -----
[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7]
----- idx -----
[[7, 0, 8, 9, 15, 10, 25, 27, 26], [22, 24, 23, 14, 17, 20, 1, 5, 4]]

========== train loader ==========

----- batch -----
tensor([18, 17, 19, 21, 26, 29, 76, 75, 71])
tensor([1, 1, 1, 2, 2, 2, 7, 7, 7])
----- batch -----
tensor([27, 22, 23, 16, 15, 13, 72, 73, 74])
tensor([2, 2, 2, 1, 1, 1, 7, 7, 7])
